# Text Summarization using Python

In [38]:
import re
import json
import pandas as pd
import numpy as np
import networkx as nx
from string import punctuation
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
from nltk import sent_tokenize, word_tokenize, pos_tag
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS as stop_words
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
class SimpleSummarize:
    def __init__(self, filename=None, k=None):
        self.txt = None
        self.word_tokens = None
        self.sent_tokens = None
        self.word_freq = None
        self.freq_dist = {}
        self.sent_scores = {}
        self.top_sents = None
        self.max_len = 40
        self.summary = ''
        self.scores = []
        self.english_stopwords = set(stopwords.words('english')) | stop_words
        if filename and k:
            self.load_file_from_disk(filename)
            self.tokenize()
            self.word_freq_dist()
            self.score_sentences()
            self.summarize(k)
    
    def load_file_from_disk(self, filename):
        with open(filename, "r") as file:
            self.txt = file.read().replace("\n", " ")
            self.txt = self.txt.replace("\'","")
    
    def tokenize(self):
        self.word_tokens = self.tokenizer(self.txt)
        #self.sent_tokens = self.simple_sent_tokenizer(self.txt)
        self.sent_tokens = sent_tokenize(self.txt)

    def simple_sent_tokenizer(self, s):
        sents = []
        for sent in s.split('.'):
            sents.append(sent.strip())
        return sents
        
    def tokenizer(self,txt):
        txt = txt.lower()
        word_tokens = word_tokenize(txt.lower())
        word_tokens = [w for w in word_tokens if w not in self.english_stopwords and re.match('[a-zA-Z-][a-zA-Z-]{2,}', w)]
        return word_tokens
    
    def word_freq_dist(self):
        self.word_freq = nltk.FreqDist(self.word_tokens)
        most_freq_count = max(self.word_freq.values())
        for k,v in self.word_freq.items():
            self.freq_dist[k] = v/most_freq_count
    
    def score_sentences(self):
        for sent in self.sent_tokens:
            words = self.tokenizer(sent)
            for word in words:
                if word.lower() in self.freq_dist.keys():
                    if len(words) < self.max_len:
                        # if key does not exist add it and the freq_dist for the first word
                        if sent not in self.sent_scores.keys():
                            self.sent_scores[sent] = self.freq_dist[word.lower()]
                        else: 
                            # the key exists and we just add the freq_dist of the following words. 
                            # We are just summing up the freq_dists for the sentence
                            self.sent_scores[sent] += self.freq_dist[word.lower()]
    
    def summarize(self, k):
        self.top_sents = Counter(self.sent_scores)
        for t in self.top_sents.most_common(k):
            self.summary += t[0].strip()+'. '
            self.scores.append((t[1],t[0]))
    

In [3]:
# foo = SimpleSummarize()
# foo.load_file_from_disk("CNNImpeachmentArticle.txt")
# foo.tokenize()
# foo.word_freq_dist()
# foo.score_sentences()
# foo.summarize(3)
# foo.summary
foo = SimpleSummarize(filename="CNNImpeachmentArticle.txt", k=3)
foo.summary

'The Judiciary Committee hearing is the latest sign that House Democrats are moving forward with impeachment proceedings against the President following the two-month investigation led by the House Intelligence Committee into allegations that Trump pushed Ukraine to investigate his political rivals while a White House meeting and $400 million in security aid were withheld from Kiev.. The House Judiciary Committee has invited President Donald Trump or his counsel to participate in the panels first impeachment hearing next week as the House moves another step closer to impeaching the President.. READ: Judiciary Chairman&#39;s invite to Trump and his lawyers to take part in upcoming impeachment hearings The hearing announcement comes as the Intelligence Committee plans to release its report summarizing the findings of its investigation to the House Judiciary Committee soon after Congress returns from its Thanksgiving recess next week.. '

## Gensim

https://towardsdatascience.com/text-summarization-in-python-76c0a41f0dc4

In [3]:
from gensim.summarization.summarizer import summarize
from gensim.test.utils import common_dictionary, common_corpus
from gensim.models import LsiModel

In [116]:
!ls

articles		   requirements.txt
CNNImpeachmentArticle.txt  simple_summarize.py
impeachment_data	   Text Summarization and Topic Modeling.ipynb
README.md


In [117]:
with open("CNNImpeachmentArticle.txt", "r") as file:
    text = file.read()
print(summarize(text))

House Judiciary Chairman Jerry Nadler sent a letter to Trump on Tuesday notifying him of the hearing and inviting the President or his counsel to participate, including asking questions of the witnesses.
The Judiciary Committee hearing is the latest sign that House Democrats are moving forward with impeachment proceedings against the President following the two-month investigation led by the House Intelligence Committee into allegations that Trump pushed Ukraine to investigate his political rivals while a White House meeting and $400 million in security aid were withheld from Kiev.
Nadler asked Trump to respond by Sunday on whether the White House wanted to participate in the hearings, as well as who would act as the President's counsel for the proceedings.


In [1]:
text2 = """We all know that music is a powerful influencer. A movie without a soundtrack doesn’t provoke the same emotional journey. A workout without a pump-up anthem can feel like a drag. But is there a way to quantify these reactions? And if so, could they be reverse-engineered and put to use?

In a new paper, researchers at the University of Southern California mapped out how things like  pitch, rhythm, and harmony induce different types of brain activity, physiological reactions (heat, sweat, and changes in electrical response), and emotions (happiness or sadness), and how machine learning could use those relationships to predict how people might respond to a new piece of music. The results, presented at a conference last week on the intersections of computer science and art, show how we may one day be able to engineer targeted musical experiences for purposes ranging from therapy to movies.

The research is part of the lab’s broader goal to understand how different forms of media, such as films and TV ads as well as music, affect people’s bodies and brains. “Once we understand how media can affect your various emotions, then we can try to productively use it for actually supporting or enhancing human experiences,” says Shrikanth Narayanan, a professor at USC and the principal investigator in the lab."""

In [4]:
summarize(text2)

'In a new paper, researchers at the University of Southern California mapped out how things like  pitch, rhythm, and harmony induce different types of brain activity, physiological reactions (heat, sweat, and changes in electrical response), and emotions (happiness or sadness), and how machine learning could use those relationships to predict how people might respond to a new piece of music.'

In [41]:
text3 = """Republican South Carolina Senator Lindsey Graham is statistically tied with Democratic challenger Jaime Harrison, with the staunch pro-Trump incumbent seeing his favorability ratings plummet among independent voters.

Graham, who was first elected to the U.S. Senate in 2002, is clinging to a 2-percentage point lead over Harrison, 47 to 45 percent, with nearly 10 percent of voters surveyed still remaining undecided about their 2020 vote. As the Change Research/Post and Courier newspaper poll notes, Graham performs poorly in hypothetical November election matchups as well as with voters who do not identify solely as Republicans. The statistical dead heat between Graham and Harrison, who has pulled in record fundraising in recent weeks, comes as the GOP senator has become one of President Donald Trump's most outspoken supporters during the ongoing impeachment inquiry.

Trump won the traditionally-Republican state of South Carolina in the 2016 presidential election by a margin of nearly 15 percentage points over Democrat Hillary Clinton.

"[Senator Lindsey Graham] looks extremely vulnerable against Democratic contender Jaime Harrison," the South Carolina pollsters noted among their key findings. "While South Carolina does not support impeaching President Trump, a majority of voters would like Senator Graham to approach the impeachment inquiry with an open mind, rather than leap to the president's defense before hearing evidence."

Graham has not always been such a staunch defender of Trump, with the longtime Republican senator infamously remarking during the 2016 primary, "If we nominate Trump, we will get destroyed......and we will deserve it."


Lindsey Graham
✔
@LindseyGrahamSC
If we nominate Trump, we will get destroyed.......and we will deserve it.

162K
1:03 PM - May 3, 2016
Twitter Ads info and privacy
143K people are talking about this
However, Graham made recent comments that he doubts the president so little that he doesn't intend on being a "fair juror," if and when the House votes to send the articles of impeachment over to the Senate.

Harrison told residents of Greenville, South Carolina on Saturday that Graham is "not worthy of this state ... the winds of change are blowing my friends," the Greenville News reported. Responding to the neck-and-neck South Carolina poll on Twitter, Harrison asked his supporters and potential voters to remember that any campaign is possible in the current political environment.

"Running against Senator Graham is a tough climb, but it's also a hill worth climbing. I've faced things folks have deemed impossible my entire life, and this is yet another journey where I prove that in America, the impossible is always possible," Harrison wrote.

Harrison declared his candidacy in June and has previously served as chair and senior counselor at the Democratic National Committee, as well as heading the Democratic Party of South Carolina. The Yale University graduate and South Carolina native was also an advisor to Congressman James Clyburn.

"It's an uphill battle, no question, but Jaime is uniquely qualified," said House Majority Whip Clyburn, told The Washington Post last week. "He has the kind of life experiences that allow him to really connect with ordinary people."

The Change Research surveyed 998 likely general election voters in the state of South Carolina between December 6-11. The poll found only 38 percent of likely voters said they will "definitely" or "probably" vote for Graham next November. Trump has higher favorability numbers than Graham, with the Republican senator holding onto the support of less than half of those surveyed. In terms of the presidential election, former Vice President Joe Biden leads the Democratic primary with 27 percent, followed by Vermont Senator Bernie Sanders with 20 percent of the vote."""

text3 = text3.strip().replace("\n"," ")

In [46]:
text3.replace("\'","")

'Republican South Carolina Senator Lindsey Graham is statistically tied with Democratic challenger Jaime Harrison, with the staunch pro-Trump incumbent seeing his favorability ratings plummet among independent voters.  Graham, who was first elected to the U.S. Senate in 2002, is clinging to a 2-percentage point lead over Harrison, 47 to 45 percent, with nearly 10 percent of voters surveyed still remaining undecided about their 2020 vote. As the Change Research/Post and Courier newspaper poll notes, Graham performs poorly in hypothetical November election matchups as well as with voters who do not identify solely as Republicans. The statistical dead heat between Graham and Harrison, who has pulled in record fundraising in recent weeks, comes as the GOP senator has become one of President Donald Trumps most outspoken supporters during the ongoing impeachment inquiry.  Trump won the traditionally-Republican state of South Carolina in the 2016 presidential election by a margin of nearly 15

## Another text summarizer

https://towardsdatascience.com/understand-text-summarization-and-create-your-own-summarizer-in-python-b26a9f09fc70

In [97]:
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx
from nltk.corpus import stopwords
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS as sw

In [35]:
def read_article(file_name):
    with open(file_name, 'r') as file:
        txt = file.read()
        sent_tokens = sent_tokenize(txt)
        return sent_tokens

In [73]:
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
    all_words = list(set(sent1 + sent2))
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
    return 1 - cosine_distance(vector1, vector2)

In [54]:
def build_similarity_matrix(sentences, stop_words):
    similarity_matrix = np.zeros([len(sentences), len(sentences)])
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2:
                continue
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)
    return similarity_matrix

In [111]:
def generate_summary(file_name, top_n=5):
    # set(stopwords.words('english')) | stop_words # Union operator for set() in python
    stop_words = set(stopwords.words('english')) | sw
    summarize_text = []
    sentences = read_article(file_name)
    sentence_similarity_matrix = build_similarity_matrix(sentences, stop_words)
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_matrix)
    scores = nx.pagerank(sentence_similarity_graph)
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    #print("Indexes of top ranked_sentence order are ", ranked_sentences)    
    for i in range(top_n):
        summarize_text.append("".join(ranked_sentences[i][1]))
    print("Summarize Text: \n", ". ".join(summarize_text))

In [74]:
s1 = 'This is a sentence that I wrote.'
s2 = 'I wrote a sentence and drew a picture.'
s3 = 'Those are some green apples.'
print(sentence_similarity(s1, s2, stop_words))
print(sentence_similarity(s1, s3, stop_words))
print(sentence_similarity(s2, s3, stop_words))

0.8760375907831333
0.7978031705815032
0.8268741925060656


In [83]:
sentences = read_article("impeachment_data/cnn_article.txt")
build_similarity_matrix(sentences, stop_words)

array([[0.        , 0.980023  , 0.93995675, ..., 0.96456677, 0.90221968,
        0.9808089 ],
       [0.980023  , 0.        , 0.97076679, ..., 0.97635149, 0.92032256,
        0.9755214 ],
       [0.93995675, 0.97076679, 0.        , ..., 0.94083432, 0.92680154,
        0.94693094],
       ...,
       [0.96456677, 0.97635149, 0.94083432, ..., 0.        , 0.92047436,
        0.96230338],
       [0.90221968, 0.92032256, 0.92680154, ..., 0.92047436, 0.        ,
        0.90988747],
       [0.9808089 , 0.9755214 , 0.94693094, ..., 0.96230338, 0.90988747,
        0.        ]])

In [118]:
generate_summary("CNNImpeachmentArticle.txt", top_n=5)

Summarize Text: 
 The resolution states that should the Trump administration refuse to cooperate in the impeachment proceedings — such as denying witnesses, which it has done — Nadler can "impose appropriate remedies, including by denying specific requests by the President or his counsel under these procedures to call or question witnesses.". The Judiciary Committee hearing is the latest sign that House Democrats are moving forward with impeachment proceedings against the President following the two-month investigation led by the House Intelligence Committee into allegations that Trump pushed Ukraine to investigate his political rivals while a White House meeting and $400 million in security aid were withheld from Kiev.. Nadler asked Trump to respond by Sunday on whether the White House wanted to participate in the hearings, as well as who would act as the President's counsel for the proceedings.. The committee announced that it would hold a hearing December 4 on the "constitutional gr

# Text Extraction from PDFs

In [ ]:
import PyPDF2

In [ ]:
pdf = 'impeachment_data/20191203_-_full_report___hpsci_impeachment_inquiry_-_20191203.pdf'
pd_file_obj = open(pdf, 'rb')
pdf_reader = PyPDF2.PdfFileReader(pd_file_obj)
num_pages = pdf_reader.getNumPages()
full_text = ''
for p in range(num_pages):
    full_text = full_text + pdf_reader.getPage(p).extractText().strip().replace('\n','')
with open('impeachment_data/trump_impeachment_inquiry.txt', 'w+') as file:
    file.write(full_text)
    file.close()

In [ ]:
full_text[:1000]

In [ ]:
impeachment_summary = SimpleSummarize(filename="impeachment_data/trump_impeachment_inquiry.txt", k=3)
impeachment_summary.summary

In [ ]:
impeachment_summary.scores

# Topic Modeling

https://nlpforhackers.io/topic-modeling/

# Topic Modeling Algorithms

There are several algorithms for doing topic modeling. The most popular ones include

LDA – Latent Dirichlet Allocation – The one we’ll be focusing in this tutorial. Its foundations are Probabilistic Graphical Models

LSA or LSI – Latent Semantic Analysis or Latent Semantic Indexing – Uses Singular Value Decomposition (SVD) on the Document-Term Matrix. Based on Linear Algebra

NMF – Non-Negative Matrix Factorization – Based on Linear Algebra

# Using Gensim for Topic Modeling

In [92]:
import re
from glob import glob
from gensim import models, corpora, similarities
from gensim.summarization.summarizer import summarize
from nltk.corpus import stopwords
from nltk import sent_tokenize, word_tokenize, pos_tag
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS as stop_words

In [99]:
def load_files_from_disk(data_dir):
    text_data_list = []
    file_list = glob(pathname=data_dir + '/*txt')
    for file in file_list: 
        with open(file, "r") as f:
            text_data_list.append(f.read())
    return text_data_list

In [225]:
class SummaryTopics:
    def __init__(self, text_data, num_topics=4):
        self.text_data = text_data
        self.num_topics = num_topics
        self.tokenized_data = []
        self.dictionary = None
        self.corpus = None
        self.lda_model = None
        self.lsi_model = None
        self.stopwords = set(stopwords.words('english')) | stop_words
    
    def text_summarize(self, text):
        text_data = text.strip().replace("\n"," ")
        summary = summarize(text_data).strip().replace("\n"," ")
        return jsonify(summary)
    
    def clean_text(self, text):
        tokenized_text = word_tokenize(text.lower())
        cleaned_text = [t for t in tokenized_text if t not in self.stopwords and re.match('[a-zA-Z\-][a-zA-Z\-]{2,}', t)]
        return cleaned_text
    
    def tokenize_text(self):
        # For gensim we need to tokenize the data and filter out stopwords
        for text in self.text_data:
            self.tokenized_data.append(self.clean_text(text))
        return self.tokenized_data
    
    def build_dictionary(self):
        # Build a Gensim Dictionary - associate word to numeric id
        self.dictionary = corpora.Dictionary(self.tokenized_data)
        return self.dictionary
    
    def build_corpus(self):
        # Transform the collection of texts to a numerical form
        self.corpus = [self.dictionary.doc2bow(text) for text in self.tokenized_data]
        return self.corpus
    
    def build_models(self):
        # Build the LDA model and the LSI model
        self.lda_model = models.LdaModel(corpus=self.corpus, 
                                         num_topics=self.num_topics, 
                                         id2word=self.dictionary)
        self.lsi_model = models.LsiModel(corpus=self.corpus, 
                                         num_topics=self.num_topics, 
                                         id2word=self.dictionary)
    
    
    

In [241]:
article_data = load_files_from_disk('impeachment_data/')
t = SummaryTopics(article_data)
tokens = t.tokenize_text()
gensim_dict = t.build_dictionary()
gensim_corpus = t.build_corpus()
t.build_models()
all_topics = []
pattern = r'"([A-Za-z0-9_\./\\-]*)"'
for idx in range(t.lda_model.num_topics):
    m = re.findall(pattern, t.lda_model.print_topic(idx,5))
    all_topics += m
for idx in range(t.lsi_model.num_topics):
    m = re.findall(pattern, t.lsi_model.print_topic(idx,5))
    all_topics += m
set(all_topics)

{'ambassador',
 'chairman',
 'committee',
 'democrats',
 'dep',
 'house',
 'impeachment',
 'president',
 'report',
 'said',
 'senate',
 'transcript',
 'trump',
 'ukraine'}

In [165]:
article_data = load_files_from_disk('impeachment_data/')
NUM_TOPICS = 5
STOPWORDS = set(stopwords.words('english')) | stop_words

In [149]:
def clean_text(text):
    tokenized_text = word_tokenize(text.lower())
    cleaned_text = [t for t in tokenized_text if t not in STOPWORDS and re.match('[a-zA-Z\-][a-zA-Z\-]{2,}', t)]
    return cleaned_text

In [166]:
# For gensim we need to tokenize the data and filter out stopwords
tokenized_data = []
for text in article_data:
    tokenized_data.append(clean_text(text))

In [167]:
# Build a Gensim Dictionary - assocation word to numeric id
dictionary = corpora.Dictionary(tokenized_data)
# for k,v in dictionary.items():
#     print((k, v))

In [168]:
# Transform the collection of texts to a numerical form
corpus = [dictionary.doc2bow(text) for text in tokenized_data]

In [169]:
# Build the LDA model
lda_model = models.LdaModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)

#Build the LSI model
lsi_model = models.LsiModel(corpus=corpus, num_topics=NUM_TOPICS, id2word=dictionary)

In [170]:
lda_topics = lda_model.print_topics()
lsi_topics = lsi_model.print_topics()

In [160]:
t = lda_topics[0][1]
pattern = r'"([A-Za-z0-9_\./\\-]*)"'
m = re.findall(pattern, t)
m

['president',
 'house',
 'ambassador',
 'trump',
 'ukraine',
 'committee',
 'sondland',
 'white',
 'security',
 'zelensky']

In [171]:
for idx in range(lda_model.num_topics):
    print(lda_model.print_topic(idx,5))

0.035*"president" + 0.020*"house" + 0.019*"ambassador" + 0.013*"ukraine" + 0.010*"committee"
0.020*"president" + 0.014*"house" + 0.014*"ambassador" + 0.013*"trump" + 0.011*"ukraine"
0.034*"president" + 0.019*"trump" + 0.018*"house" + 0.016*"ambassador" + 0.013*"ukraine"
0.024*"president" + 0.015*"ambassador" + 0.013*"house" + 0.012*"trump" + 0.010*"ukraine"
0.047*"president" + 0.016*"house" + 0.016*"trump" + 0.012*"ambassador" + 0.011*"sondland"


In [172]:
for idx in range(lsi_model.num_topics):
    print(lda_model.print_topic(idx,5))

0.035*"president" + 0.020*"house" + 0.019*"ambassador" + 0.013*"ukraine" + 0.010*"committee"
0.020*"president" + 0.014*"house" + 0.014*"ambassador" + 0.013*"trump" + 0.011*"ukraine"
0.034*"president" + 0.019*"trump" + 0.018*"house" + 0.016*"ambassador" + 0.013*"ukraine"
0.024*"president" + 0.015*"ambassador" + 0.013*"house" + 0.012*"trump" + 0.010*"ukraine"
0.047*"president" + 0.016*"house" + 0.016*"trump" + 0.012*"ambassador" + 0.011*"sondland"


In [158]:
print("=" * 115)
print("LDA Model:")
for idx in range(NUM_TOPICS):
    print("Topic #%s:"%idx, lda_model.print_topic(idx,5))
print("=" * 115)
print("LSI Model:")
for idx in range(NUM_TOPICS):
    print("Topic #%s:"%idx, lsi_model.print_topic(idx,5))
print("=" * 115)

LDA Model:
Topic #0: 0.039*"president" + 0.017*"house" + 0.014*"ambassador" + 0.013*"trump" + 0.013*"ukraine"
Topic #1: 0.031*"president" + 0.017*"ambassador" + 0.017*"house" + 0.016*"trump" + 0.013*"ukraine"
Topic #2: 0.042*"president" + 0.016*"trump" + 0.016*"house" + 0.013*"ukraine" + 0.013*"ambassador"
Topic #3: 0.026*"president" + 0.020*"house" + 0.019*"ambassador" + 0.015*"trump" + 0.011*"white"
Topic #4: 0.041*"president" + 0.019*"house" + 0.016*"ambassador" + 0.012*"trump" + 0.010*"ukraine"
Topic #5: 0.038*"president" + 0.018*"house" + 0.015*"ambassador" + 0.014*"trump" + 0.011*"ukraine"
Topic #6: 0.040*"president" + 0.017*"ambassador" + 0.015*"house" + 0.014*"trump" + 0.011*"ukraine"
Topic #7: 0.036*"president" + 0.015*"house" + 0.013*"trump" + 0.012*"ambassador" + 0.012*"committee"
Topic #8: 0.033*"president" + 0.018*"trump" + 0.015*"house" + 0.013*"ukraine" + 0.011*"ambassador"
Topic #9: 0.024*"president" + 0.016*"house" + 0.014*"ambassador" + 0.012*"committee" + 0.010*"ukra

In [129]:
t = """
The report puts Trump personal lawyer Rudy Giuliani at the center of a scheme to force out the U.S. ambassador to Ukraine and pressure that country’s government to investigate Joe Biden’s family and a conspiracy theory that Ukraine interfered in the 2016 U.S. election.
The House obtained AT&T call records showing Giuliani in contact with phone numbers associated with the White House, the Office of Management and Budget, top Intelligence Committee Republican Devin Nunes, and Giuliani associate Lev Parnas. The report doesn’t say who in the White House or OMB participated in the calls.
The calls and texts were made during the time period when Giuliani was publicly discussing his efforts to pursue investigations into the Bidens and a conspiracy theory about Ukrainian interference in the 2016 election.
House Intelligence Chairman Adam Schiff said the call records show that “there was considerable coordination among the parties including the White House” in a smear campaign against then-U.S. Ambassador Marie Yovanovitch.
The committee also found Giuliani in contact on Aug. 8 with phone numbers associated with the White House amid negotiations with Ukrainian officials about announcing investigations. The records also showed European Union Ambassador Gordon Sondland in contact with White House and OMB phone numbers on Aug. 9.
One of the Sondland calls came minutes before a text message he sent saying that he thought Trump strongly wanted the “deliverable.” Sondland later said that referred to an announcement by Ukraine of investigations sought by Trump and Giuliani.
"""
bow = dictionary.doc2bow(clean_text(t))
print(lsi_model[bow])
print(lda_model[bow])

[(0, 8.21516815161783), (1, -3.292904335916135), (2, 5.979631898609965), (3, -4.467286967563506)]
[(1, 0.9923591)]


In [130]:
lda_index = similarities.MatrixSimilarity(lda_model[corpus])

In [131]:
sims = lda_index[lda_model[bow]]

In [132]:
sims = sorted(enumerate(sims), key=lambda item: -item[1])

In [133]:
print(sims[:10])

[(2, 0.99994105), (0, 0.9488427), (3, 0.78462344), (1, 0.0269688)]


In [134]:
document_id, similarity = sims[2]
print(article_data[document_id][:1000])

The House Intelligence Committee voted along party lines Tuesday night to approve a report that found evidence of President Donald Trump's misconduct and obstruction of Congress is "overwhelming."

The report, released earlier Tuesday, will form the backbone of the impeachment proceedings against the President and charges that Trump's conduct toward Ukraine compromised national security
The vote will send the report to the House Judiciary Committee as that panel considers moving forward on articles of impeachment.
The 300-page report from the House Intelligence Committee sets the stage for the impeachment of a US president for just the third time in history. The report stops short of outright recommending impeachment, with House Intelligence Chairman Adam Schiff and the other committee chairs leading the inquiry said that was a decision Congress ultimately has to make.
But the report makes clear that's where Democrats are heading.
"I'm going to reserve any kind of a public judgment on 

# Using Scikit-Learn for Topic Modeling

scikit-learn offers an NMF model in addition to LDA and LSI models

In [135]:
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer

In [136]:
NUM_TOPICS = 5

vectorizer = CountVectorizer(min_df=3, max_df=0.9, stop_words='english', lowercase=True,
                            token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(article_data)

In [137]:
# https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html
lda_model = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=10, learning_method='online')
lda_Z = lda_model.fit_transform(data_vectorized)
print(lda_Z.shape) # tuple (num_docs, num_topics)

(4, 5)


In [138]:
nmf_model = NMF(n_components=NUM_TOPICS)
nmf_Z = nmf_model.fit_transform(data_vectorized)
print(nmf_Z.shape)

(4, 5)


In [139]:
lsi_model = TruncatedSVD(n_components=NUM_TOPICS)
lsi_Z = lsi_model.fit_transform(data_vectorized)
print(lsi_Z.shape)

(4, 4)


In [140]:
print(lda_Z[0])
print(nmf_Z[0])
print(lsi_Z[0])

[9.94119683e-01 5.81994039e-03 1.99707494e-05 2.01479215e-05
 2.02581617e-05]
[12.01307451  0.23971834  0.63937743  2.77510201  0.18146798]
[ 9.65716426e+02 -1.65177931e+02 -1.40420057e-01 -1.29127866e-01]


In [141]:
def print_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])
 
print("LDA Model:")
print_topics(lda_model, vectorizer)
print("=" * 20)
 
print("NMF Model:")
print_topics(nmf_model, vectorizer)
print("=" * 20)
 
print("LSI Model:")
print_topics(lsi_model, vectorizer)
print("=" * 20)

LDA Model:
Topic 0:
[('volker', 273.75350024654267), ('zelensky', 222.4418278789622), ('meeting', 204.53130698940905), ('affairs', 191.9620358494488), ('oversight', 164.8790887700714), ('taylor', 150.03198368826347), ('document', 140.43942212714282), ('testified', 130.9193321179681), ('deputy', 125.88981557658056), ('engel', 121.96983487207848)]
Topic 1:
[('zelensky', 258.14923139154865), ('meeting', 141.64499688030472), ('military', 99.08220963806107), ('united', 95.93989188274847), ('announcement', 82.66303582246141), ('volker', 72.5270192543754), ('aid', 66.68316674126515), ('reelection', 63.806254672422305), ('conditioned', 60.49045812440639), ('subpoenas', 60.440916417512526)]
Topic 2:
[('zelensky', 1.0134425160838034), ('volker', 0.9329729667156315), ('affairs', 0.8758248240122222), ('document', 0.849617806457963), ('did', 0.8302887122827746), ('august', 0.8252802023460254), ('meeting', 0.8050350026983764), ('taylor', 0.7883426230620103), ('testified', 0.7653160362257991), ('morr

In [142]:
text = "The President is underfire for his corrupt behavior. Trump Zelensky impeachment democrats volker"
x = nmf_model.transform(vectorizer.transform([text]))[0]
print(x)

[0.00881501 0.0461733  0.00893442 0.         0.        ]


In [144]:
import pandas as pd
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, LabelSet
output_notebook()

svd = TruncatedSVD(n_components=2)
documents_2d = svd.fit_transform(data_vectorized)
 
df = pd.DataFrame(columns=['x', 'y', 'document'])
df['x'], df['y'], df['document'] = documents_2d[:,0], documents_2d[:,1], range(len(article_data))
 
source = ColumnDataSource(ColumnDataSource.from_df(df))
labels = LabelSet(x="x", y="y", text="document", y_offset=8,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
 
plot = figure(plot_width=600, plot_height=600)
plot.circle("x", "y", size=12, source=source, line_color="black", fill_alpha=0.8)
plot.add_layout(labels)
show(plot, notebook_handle=True)


Loading BokehJS ...

In [145]:
svd = TruncatedSVD(n_components=2)
words_2d = svd.fit_transform(data_vectorized.T)
df = pd.DataFrame(columns=['x', 'y', 'document'])
df['x'], df['y'], df['word'] = words_2d[:,0], words_2d[:,1], vectorizer.get_feature_names()
source = ColumnDataSource(ColumnDataSource.from_df(df))
labels = LabelSet(x="x", y="y", text="word", y_offset=8,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
plot = figure(plot_width=600, plot_height=600)
plot.circle("x", "y", size=12, source=source, line_color="black", fill_alpha=0.8)
plot.add_layout(labels)
show(plot, notebook_handle=True)

## More about Latent Dirichlet Allocation


LDA is the most popular method for doing topic modeling in real-world applications. That is because it provides accurate results, can be trained online (do not retrain every time we get new data) and can be run on multiple cores. Let’s repeat the process we did in the previous sections with sklearn and LatentDirichletAllocation.

In [149]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD

In [177]:
NUM_TOPICS = 5

vectorizer = CountVectorizer(min_df=3, max_df=0.9, stop_words='english', lowercase=True, 
                             token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(article_data)
                             
# Build a Latent Dirichlet Allocation Model
lda_model = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=10, learning_method='online')
lda_Z = lda_model.fit_transform(data_vectorized)
 
text = "The President is underfire for his corrupt behavior. Trump Zelensky impeachment democrats volker"
x = lda_model.transform(vectorizer.transform([text]))[0]
# the largest topic contributes more to the document than the smaller topics
print(x, x.sum())

[0.05080756 0.0507848  0.40283253 0.05192533 0.44364977] 1.0


In [179]:
import pyLDAvis.sklearn
 
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
panel

/home/john/anaconda3/envs/nlp/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
4     -119.746765  -56.879826       1        1  59.727028
3      -63.186993   56.362408       2        1  38.713797
2       50.537365  111.941399       3        1   1.549658
1       93.300476  -11.830882       4        1   0.004760
0        3.660819 -100.682213       5        1   0.004757, topic_info=    Category        Freq       Term       Total  loglift  logprob
357  Default  426.000000     volker  426.000000  30.0000  30.0000
374  Default  613.000000   zelensky  613.000000  29.0000  29.0000
211  Default  430.000000    meeting  430.000000  28.0000  28.0000
8    Default  274.000000    affairs  274.000000  27.0000  27.0000
242  Default  257.000000  oversight  257.000000  26.0000  26.0000
..       ...         ...        ...         ...      ...      ...
234   Topic5    0.004158        omb  124.988312  -0.3576  -5.2697
120   Topic5    0.004231      eliot  131.084658  -0.3880  -5.2525
304   Topic5    0.004639       sent  170.582772  -0.5592  -5.1603
165   Topic5    0.004165     holmes  130.508359  -0.3993  -5.2682
347   Topic5    0.004519     united  238.816582  -0.9218  -5.1865

[350 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
0         1  0.371979    accept
0         2  0.619964    accept
2         1  0.181537   accused
2         2  0.726148   accused
4         1  0.277869      acts
...     ...       ...       ...
373       2  0.096679      york
373       3  0.019336      york
374       1  0.479462  zelensky
374       2  0.513709  zelensky
374       3  0.006523  zelensky

[432 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 3, 2, 1])